In [12]:
from google.colab import files
uploaded = files.upload()

Saving wells co c.xlsx to wells co c (2).xlsx


In [19]:
import pandas as pd
from datetime import datetime

def read_excel_file(file_path, sheet_name):
    df = pd.read_excel(file_path, sheet_name=sheet_name)
    return df

def wisconsin_wells_list(file_path, sheet_name):
    df = read_excel_file(file_path, sheet_name)
    well_id_list = df["WI_UNIQUE_"]
    return well_id_list

def radium_concentration_rate(file_path, sheet_name):
    df = read_excel_file(file_path, sheet_name)
    well_concentration_list = df["Measured A"]
    return well_concentration_list

def formatted_sample_dates(file_path, sheet_name):
    df = read_excel_file(file_path, sheet_name)
    sample_date_list = df["Sample Date"]
    return sample_date_list

def wells_sampled_once_per_year(file_path, sheet_name):
    well_ids = wisconsin_wells_list(file_path, sheet_name)
    concentrations = radium_concentration_rate(file_path, sheet_name)
    dates = formatted_sample_dates(file_path, sheet_name)

    # Create a dictionary to count the samples per well per year
    samples_once_per_year = {}
    concentration_dict = {}

    for well_id, date, conc in zip(well_ids, dates, concentrations):
        year = pd.to_datetime(date).year
        if (well_id, year) in samples_once_per_year:
            samples_once_per_year[(well_id, year)] += 1
        else:
            samples_once_per_year[(well_id, year)] = 1
            concentration_dict[(well_id, year)] = conc

    # Now filter the dictionary to find wells with only one sample per year
    wells_sampled_once = [(well_id, year, concentration_dict[(well_id, year)])
                          for (well_id, year), count in samples_once_per_year.items() if count == 1]

    return wells_sampled_once

def wells_sampled_multiple_per_year(file_path, sheet_name):
    well_ids = wisconsin_wells_list(file_path, sheet_name)
    concentrations = radium_concentration_rate(file_path, sheet_name)
    dates = formatted_sample_dates(file_path, sheet_name)

    # Create a dictionary to count the samples per well per year
    samples_multiple_per_year = {}
    concentration_dict = {}

    for well_id, date, conc in zip(well_ids, dates, concentrations):
        year = pd.to_datetime(date).year
        if (well_id, year) in samples_multiple_per_year:
            samples_multiple_per_year[(well_id, year)] += 1
            # Here check if new concentration is greater than the current one
            if concentration_dict[(well_id, year)] < conc:
                concentration_dict[(well_id, year)] = conc
        else:
            samples_multiple_per_year[(well_id, year)] = 1
            concentration_dict[(well_id, year)] = conc

    # Now filter the dictionary to find wells with only one sample per year
    wells_sampled_multiple = [(well_id, year, concentration_dict[(well_id, year)])
                              for (well_id, year), count in samples_multiple_per_year.items() if count > 1]

    return wells_sampled_multiple


def combine_wells_sampled_multiple(file_path, sheet_name):
    all_wells = wells_sampled_once_per_year(file_path, sheet_name)
    wells_sampled_multiple = wells_sampled_multiple_per_year(file_path, sheet_name)

    # Read the original dataframe
    df = read_excel_file(file_path, sheet_name)

    # Filter the rows based on the combined wells data
    filtered_df = df[(df["WI_UNIQUE_"].isin([well[0] for well in all_wells])) |
                     ((df["WI_UNIQUE_"].isin([well[0] for well in wells_sampled_multiple])) &
                      (df["Sample Date"].apply(lambda x: pd.to_datetime(x).year).isin([well[1] for well in wells_sampled_multiple])))]

    return filtered_df

def generate_combined_wells_spreadsheet(file_path, sheet_name, output_file_path):
    filtered_df = combine_wells_sampled_multiple(file_path, sheet_name)

    # Save the filtered dataframe to an Excel file
    filtered_df.to_excel(output_file_path, index=False)

def main():
    file_path = "wells co c (2).xlsx"
    sheet_name = "Sheet1"
    output_file_path = "combined_wells_data.xlsx"

    generate_combined_wells_spreadsheet(file_path, sheet_name, output_file_path)

    wells_data_frame = read_excel_file(file_path, sheet_name)
    print(wells_data_frame)

    wells_list = wells_sampled_once_per_year(file_path, sheet_name)
    print(len(wells_list))
    print(wells_list, end='')

    print()
    wells_sampled_multiple_times = wells_sampled_multiple_per_year(file_path, sheet_name)
    print(len(wells_sampled_multiple_times))
    print(wells_sampled_multiple_times)

    combined_wells = combine_wells_sampled_multiple(file_path, sheet_name)
    print(len(combined_wells))
    print(combined_wells)


if __name__ == "__main__":
    main()


    WI_UNIQUE_  Measured A Sample Date
0       AAA484       0.000  2022-10-24
1       AAA484       0.000  2020-08-13
2       AAA510       2.200  2020-01-21
3        AC715       7.870  2013-08-08
4        AJ774       7.070  2012-08-06
..         ...         ...         ...
497      YM053       1.300  2014-12-17
498      YN611       1.502  2015-12-07
499      YQ304       1.980  2016-04-20
500      ZR344       9.970  2019-04-15
501      ZR344      10.770  2017-12-18

[502 rows x 3 columns]
350
[('AAA484', 2022, 0.0), ('AAA484', 2020, 0.0), ('AAA510', 2020, 2.2), ('AC715', 2013, 7.87), ('AJ774', 2012, 7.07), ('AX007', 2023, 7.32), ('AX007', 2022, 6.8), ('AX007', 2021, 7.47), ('AX007', 2020, 6.0), ('AX007', 2019, 7.71), ('AX007', 2018, 6.65), ('AX007', 2017, 6.48), ('AX007', 2016, 8.3), ('AX007', 2015, 8.07), ('AX007', 2014, 7.46), ('AX007', 2013, 7.67), ('AY338', 2006, 6.0), ('AY377', 2023, 6.26), ('AY377', 2022, 6.14), ('AY377', 2021, 7.85), ('AY377', 2020, 7.18), ('AY377', 2019, 6.82), (